In [1]:
import pandas as pd
import numpy as np
from itertools import product

In [10]:
element_count = 7

In [11]:
def generate_combinations(element_count, total_sum):

    # Since the ratios are 1:2:3:4, we look for combinations of these numbers
    ratios = [1, 2, 3, 4]
    
    # Generate all possible products of these ratios for the given number of elements
    possible_combinations = product(ratios, repeat=element_count)
    
    # Filter combinations that sum up to the total sum
    valid_combinations = [combo for combo in possible_combinations if sum(combo) == total_sum]
    
    return valid_combinations

# Function to group numbers until they add up to 4
def group_numbers(pattern):
    groups = []
    current_group = []
    current_sum = 0

    for num in pattern:
        if current_sum + num > 4:
            groups.append(current_group)
            current_group = [num]
            current_sum = num
        else:
            current_group.append(num)
            current_sum += num

    groups.append(current_group)
    return groups

# Function to generate the pattern of silence
def silence_pattern(pattern):
    if np.isnan(pattern).any():
        return []
    return [num - 1 for num in pattern if num > 0]


In [12]:
df = pd.DataFrame(generate_combinations(element_count=element_count, total_sum=12))

column_len = len(df.columns)   

# show the pattern in the 6th column
df['pattern'] = df.apply(lambda x: ''.join(str(x[i]) for i in range(column_len)), axis=1)

# Convert patterns from strings to lists of integers
df['pattern'] = df['pattern'].apply(lambda pattern: [int(num) for num in str(pattern)])

# Apply the function to each pattern in the DataFrame
df['groups'] = df['pattern'].apply(group_numbers)

# Label patterns as "simple" or "complex"
df['label'] = df['groups'].apply(lambda groups: 'simple' if all(sum(group) == 4 for group in groups) else 'complex')

In [13]:
# Create a DataFrame for simple patterns
simple_df = df[df['label'] == 'simple'].reset_index(drop=True)

# Create a DataFrame for complex patterns
complex_df = df[df['label'] == 'complex'].reset_index(drop=True)

simple_df = simple_df.drop(columns=['groups', 'label'])
complex_df = complex_df.drop(columns=['groups', 'label'])

# Concatenate simple and complex patterns into their own columns in a new DataFrame
combo_chart = pd.concat([simple_df['pattern'], complex_df['pattern']], axis=1)
combo_chart.columns = ['simple', 'complex']

# Apply the function to the 'simple' and 'complex' columns
combo_chart['simple_silence'] = combo_chart['simple'].apply(silence_pattern)
combo_chart['complex_silence'] = combo_chart['complex'].apply(silence_pattern)

combo_chart = combo_chart[['simple', 'simple_silence', 'complex', 'complex_silence']]

# Save the DataFrame to a CSV file
combo_chart.to_csv(f'combo_chart_{element_count}.csv', index=False)